In [ ]:
import re
import string
import scipy
import pickle
import pandas as pd
import numpy as np
import re
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as Log_Reg
from sklearn.ensemble import GradientBoostingClassifier as GRAD_BOOST
from sklearn.ensemble import RandomForestClassifier as RFCL
#pip install scikit-learn-extra
#from sklearn_extra.cluster import KMedoids
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import os
import joblib
from nltk.corpus import stopwords
import nltk

## Dataframe

In [ ]:
female = []
for root, dirs, files_read in os.walk(r'C:\Users\Ansar\Desktop\New folder\corpus\female'):
    for file in files_read:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'rb') as f:
                data = f.read()
                female.append(data)
df_female= pd.DataFrame(female)
df_female=df_female.rename(columns={0: 'tweet'})
df_female['label']='female'


male = []
for root, dirs, files_read in os.walk(r'C:\Users\Ansar\Desktop\New folder\corpus\male'):
    for file in files_read:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'rb') as f:
                data = f.read()
                male.append(data)
df_male= pd.DataFrame(male)
df_male=df_male.rename(columns={0: 'tweet'})

df_male['label']='male'
df = pd.concat([df_male, df_female], axis=0,ignore_index=True)
df = df.sample(frac = 1)
df=df.reset_index(drop=True)
df

,tweet,label
0,"b'Frances Grau (11.09.2013) on mass media, soc...",male
1,b'#Jotakeirabaziarte @minte1974\r\nZorionak! @...,male
2,"b'Editing Dairy Goat Journal. Finally, somethi...",male
3,b'Have you used a Venn Diagram to differentiat...,female
4,b'Have some fun &amp; help save the Earth! Yik...,male
...,...,...
420,b'Mejora tu marketing web online monitorizando...,female
421,"b'Dear friend,\nLike my Facebook Page\nhttp://...",male
422,b'#PAC-12 has strong presence on preseason col...,male
423,b'Listening to @clsaarinen&#39;s wonderful pre...,female


## Data cleaning

In [ ]:
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))
data = [] 
for lp in range(0, 425):
    clean_data = re.sub('[^a-zA-Z]', ' ', str((df['tweet'][lp])))
    clean_data.lower()
    clean_data = clean_data.split()

    clean_data=[word for word in clean_data if not word in stop_words]
    clean_data = ' '.join(clean_data)
    data.append(clean_data)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ansar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Stylometry Based Features

In [ ]:
stylo_lstt=[':','#','([0-9])','\\(','\\)','=','-','\?','%','&','.','_','!','([A-Z]*)',';',',',' ','([a-z]*)','/','@']

stylo_list=[]
for outer_loop in stylo_lstt:
    for inner_loop in range(0,len(df)):
            itering = re.findall(outer_loop, str(df['tweet'][inner_loop]))
            counting= len(itering)
            stylo_list.append(counting)
rows = 425
chunks = [stylo_list[itr:itr+rows] for itr in range(0, len(stylo_list), rows)]
chunks=np.array(chunks)
chunks=chunks.T
df_stylo=pd.DataFrame(chunks,columns=['Colon','Sharp','Digits','Opening_bracket',
                                 'Closing_bracket','Equal','Dash','Question_mark',
                                 'Percentage','Ampersand','Full_stop','Underscore',
                                 'Exclamation_mark','Capital_letter','Semi_colon',
                                 'Comma','Space','Small_letters','Slash','At_sign'])
df_stylo

,Colon,Sharp,Digits,Opening_bracket,Closing_bracket,Equal,Dash,Question_mark,Percentage,Ampersand,Full_stop,Underscore,Exclamation_mark,Capital_letter,Semi_colon,Comma,Space,Small_letters,Slash,At_sign
0,630,534,3598,52,56,48,660,118,22,902,67139,142,74,66496,910,234,5758,32364,1466,558
1,188,540,2754,96,98,22,274,174,28,702,83837,148,118,82806,708,348,10644,41534,428,692
2,1104,945,4970,7,7,73,1797,143,90,1081,126763,741,143,125668,1085,210,11484,62616,3218,227
3,690,437,1867,9,42,0,54,110,11,456,58677,64,74,57957,460,172,6626,29817,1103,229
4,116,174,324,0,2,16,42,20,2,150,10827,14,38,10660,150,0,860,5656,430,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,102,46,388,18,50,0,70,10,2,68,9447,28,36,9346,68,4,826,4518,194,202
421,1263,82,39708,40,102,18,1414,112,7,1167,236601,21,1392,235469,1168,241,9801,162798,3492,101
422,161,428,1451,20,22,44,484,194,1,479,52793,70,60,52046,483,334,6615,26325,468,394
423,1268,480,4317,88,144,95,870,166,6,1430,119008,53,209,117475,1434,250,11486,58762,3103,455


In [ ]:
df_stylo[::].values

array([[  630,   534,  3598, ..., 32364,  1466,   558],
       [  188,   540,  2754, ..., 41534,   428,   692],
       [ 1104,   945,  4970, ..., 62616,  3218,   227],
       ...,
       [  161,   428,  1451, ..., 26325,   468,   394],
       [ 1268,   480,  4317, ..., 58762,  3103,   455],
       [   68,     9,   216, ...,  2590,   160,    11]])

In [ ]:
y=df['label']

## Splitting for Stylometry based features

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_stylo[::].values, y, train_size=0.8,random_state=42)

## RFC for Stylometry based features

In [ ]:
rfclass = RFCL(max_depth=2,criterion='gini',max_features='log2',n_estimators=500)
rfclass.fit(X_train, y_train)
prediction_rfc = rfclass.predict(X_test)
score = metrics.accuracy_score(y_test, prediction_rfc)
print("accuracy:",score*100)

accuracy: 67.05882352941175


## Content Based Methods (TFIDF,CountVectorizer)

In [ ]:
tfidf_vect = TfidfVectorizer(max_features=3000, analyzer='char',ngram_range=(2,3),stop_words='english') #CountVectorizer
matrix = tfidf_vect.fit_transform(data)
matrix=matrix.toarray()

In [ ]:
matrix

array([[0.08018937, 0.00497391, 0.00372127, ..., 0.01363599, 0.        ,
        0.00105585],
       [0.07935589, 0.00233412, 0.00120057, ..., 0.00146644, 0.0004769 ,
        0.00136258],
       [0.08195314, 0.00368827, 0.00758837, ..., 0.00210654, 0.00205519,
        0.00065245],
       ...,
       [0.10167476, 0.00243191, 0.00218903, ..., 0.00038197, 0.00223594,
        0.00035492],
       [0.08316931, 0.00290415, 0.00311203, ..., 0.00152047, 0.00089004,
        0.00183662],
       [0.06281336, 0.        , 0.        , ..., 0.        , 0.00335204,
        0.00319246]])

## Splitting for Content based features

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(matrix, y, train_size=0.8,random_state=42)

## RFC for content based methods

In [ ]:
rfclass_content = RFCL(max_depth=2,criterion='gini',max_features='sqrt',
                       n_estimators=500,class_weight='balanced').fit(X_train, y_train)
prediction_rfc_content = rfclass_content.predict(X_test)
score = metrics.accuracy_score(y_test, prediction_rfc_content)
print("accuracy:",score*100)

accuracy: 70.58823529411765


## Gradient Boosting Classifier Content Based 

In [ ]:
clf_gbc = GRAD_BOOST(n_estimators=1000, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)
clf_gbc.score(X_test, y_test)*100

81.17647058823529

## Logistic Reggression Content Based

In [ ]:
LR=Log_Reg(class_weight='dict',solver='sag',max_iter=500).fit(X_train,y_train)
LR.score(X_test,y_test)*100

55.294117647058826

## Application

In [ ]:
clf_gbc_app = GRAD_BOOST(n_estimators=1000, learning_rate=1.0,max_depth=1, random_state=0).fit(matrix, y)

## Saving the model into disk

In [ ]:
file_model = 'clf_gbc_model.sav'
joblib.dump(clf_gbc_app, file)

['gbc_clf.sav']

## Input new data

In [ ]:
test_data=input()

hello 44jkn eo84 ng45o5 


## Cleaning, Vectorization, Loading the model and Prediction 

In [ ]:
tdata={'con':[test_data]}
df_test=pd.DataFrame(tdata)

test_data = [] 
for lp in range(0, len(df_test)):
    clean_data = re.sub('[^a-zA-Z]', ' ', str((df_test['con'][lp])))
    clean_data.lower()
    clean_data = clean_data.split()

    clean_data=[word for word in clean_data if not word in stop_words]
    clean_data = ' '.join(clean_data)
    test_data.append(clean_data)

mat_test = tfidf_vect.transform(test_data)

loaded_model = joblib.load(file_model)
result = loaded_model.predict(mat_test[[0]])
print(result)

['female']


In [ ]:
stop_words = set(stopwords.words('english')) 

clean_data_test=[]
for tok in range(0,len(df_test)):

    tweet2 = re.sub(r'^RT[\s]+', '', str(df_test['con'][tok]))
    tweet2=tweet2.lower()
    # remove digits
    tweet2 = ''.join([i for i in tweet2 if not i.isdigit()])
    # remove punctuation
    tweet2=re.sub(r'[^\w]', ' ', tweet2)
    # remove words whose length is 3
    tweet2= re.sub(r'\b\w{1,2}\b', '', tweet2)
    tweet2=tweet2.split()
    # Removing stop words
    tweet2 = [word for word in tweet2 if not word in stop_words]
    tweet2 = ' '.join(tweet2)
    clean_data_test.append(tweet2)

In [ ]:
mat_test12 = tfidf_vect.transform(test_data)

loaded_model = joblib.load(file_model)
result = loaded_model.predict(mat_test12[[0]])
print(result)

['female']
